In [ ]:
data_dir = 'data'

# pytorch_mnist.ipynb
# WESmith 06/26/23
## Review of MNIST processing with pytorch.
## see https://nextjournal.com/gkoehler/pytorch-mnist

## 06/30/23 added capability to look at an embedding space in a modified network

In [ ]:
import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim
import torchvision
import numpy               as np
import matplotlib.pyplot   as plt
from sklearn.manifold import TSNE

In [ ]:
n_epochs         = 3
batch_size_train = 64
batch_size_test  = 1000
learning_rate    = 0.01
momentum         = 0.5
log_interval     = 10  # WS (was 10)

random_seed      = 42
torch.backends.cudnn.enabled = False  # disable nondeterministic algorithms
_ = torch.manual_seed(random_seed)

In [ ]:
#torchvision.datasets.MNIST?

In [ ]:
#torch.utils.data.DataLoader?

In [ ]:
train_data = torchvision.datasets.MNIST(data_dir, train=True, download=True,
                                         transform=torchvision.transforms.Compose([
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize(
                                             (0.1307,), (0.3081))]))  # predefined global mean, std

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size_train, shuffle=True)

In [ ]:
test_data  = torchvision.datasets.MNIST(data_dir, train=False, download=True,
                                         transform=torchvision.transforms.Compose([
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize(
                                             (0.1307,), (0.3081))]))  # predefined global mean, std

In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [ ]:
train_data.data.shape, train_data.targets.shape, test_data.data.shape, test_data.targets.shape

In [ ]:
examples = enumerate(test_loader)  # create an emumerate object: it yields a tuple of (count, object)

In [ ]:
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape, example_targets.shape

In [ ]:
# get the full test set for scatterplots below
test_loader_full = torch.utils.data.DataLoader(test_data, batch_size=10000, shuffle=True)

In [ ]:
_, (test_data_full, test_targ_full) = next(enumerate(test_loader_full))

In [ ]:
test_data_full.shape, test_targ_full.shape

In [ ]:
n_epochs      = 6
train_losses  = []
train_counter = []
test_losses   = []

In [ ]:
test_counter  = [i*len(train_loader.dataset) for i in range(1, n_epochs + 1)]

In [ ]:
test_counter

In [ ]:
# WS original model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d( 1, 10, kernel_size=5) #  1 channel coming in,  10 channels out
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 10 channels coming in, 20 channels out
        self.conv2_drop = nn.Dropout2d()
        self.fc1   = nn.Linear(320, 50) # fully connected 4 x 4 x 20 = 320 inputs, 50 outputs
        self.fc2   = nn.Linear( 50, 10) # fully connected, 50 inputs, 10 outputs, 1 for each integer
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)  # self.training: Boolean, sets training or eval mode
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # WS added dim=1 to avoid throwing warnings

In [ ]:
def train(epoch, log_interval=100):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss   = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            ll = len(train_loader.dataset)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), ll,
                  100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx * 64) + ((epoch - 1) * ll))
            torch.save(network.state_dict(),   'results/model.pth')
            torch.save(optimizer.state_dict(), 'results/optimizer.pth')

In [ ]:
def test():
    network.eval()
    test_loss = 0
    correct   = 0
    with torch.no_grad():
        for data, target in test_loader:
            output     = network(data)
            # nll: negative log likelihood, size_average is deprecated, 'reduction' default is 'mean'
            test_loss += F.nll_loss(output, target, reduction='sum').item() #size_average=False).item()
            pred       = output.data.max(1, keepdim=True)[1]
            correct   += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))

In [ ]:
# WS create a precision matrix showing confusion errors
def precision(loader):  # loader is train_loader or test_loader
    dd = torch.zeros(10, 10) # row is truth, col is prediction
    network.eval()
    with torch.no_grad():
        for data, target in loader:
            output = network(data)
            for k in range(len(output)):
                pred  = output[k].argmax().item()
                truth = target[k].item()
                dd[truth, pred] += 1
    count = 0
    for k in range(10):
        count += dd[k, k]
    print(f'accuracy = {100. * count / dd.sum():.4f}')
    return dd.numpy()

In [ ]:
network   = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
for epoch in range(1, n_epochs + 1):
    train(epoch, 100)
    test()

In [ ]:
fig = plt.figure(figsize=(14, 6))
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.grid()
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

In [ ]:
test_counter, test_losses

In [ ]:
def get_intermediate(model, name, input_tensor):
    '''
    access the output of an intermediate layer of a network
    model: name of the network (str)
    name:  name of the intermediate layer (str)
           note: all layer names of a model can be seen by typing the model's name in a cell
                 and executing that cell
    '''
    intermed = {}
    def get_inter(name):
        nonlocal intermed
        def hook(model, input, output):
            intermed[name] = output.detach()
        return hook
    
    try:
        # WS is there a simpler way to isolate a layer?
        layer = list(model.modules())[0].__dict__['_modules'][name]
    except:
        print(f'layer {layer} not found')
        return None
    
    hook_handle   = layer.register_forward_hook(get_inter(name))
    output        = model(input_tensor)
    hook_handle.remove()
    
    dd = intermed[name]
    print(f'{name}: {layer}, with shape {list(dd.shape)} and size {dd.numel()}')
    
    return dd

In [ ]:
network

In [ ]:
example_data.shape

In [ ]:
# get an intermediate layer
upper = 5000 # 10000 is max of test data
layer = 'fc2'
out   = get_intermediate(network, layer, test_data_full[0:upper])

In [ ]:
out.shape

In [ ]:
norm_vectors = out/out.mean(dim=1, keepdim=True)

In [ ]:
# now compress the space to 2D for viewing
pts = TSNE(n_components=2, perplexity=10, verbose=1).fit_transform(norm_vectors)

In [ ]:
targs = test_targ_full[0:upper].numpy()
class_colors = {0:'r', 1:'g', 2:'b', 3:'c', 4:'m', 5:'y', 6:'w', 
                7:'#BBBBBB', 8:'#FFAAAA', 9:'#AAFFAA'}

legend_handles = []
legend_labels  = []
for label, color in class_colors.items():
    legend_handles.append(plt.Line2D([], [], marker='o', color='k', markerfacecolor=color, markersize=10))
    legend_labels.append(label)

fig, ax = plt.subplots(figsize=(12, 12))
#plt.figure(figsize=(14, 14))
fig.patch.set_facecolor('#333333')
ax.set_facecolor('black')
ax.scatter(pts[:,0], pts[:,1], s=10, c=[class_colors[label] for label in targs])
#for j, k in enumerate(targs):
#    plt.annotate(str(k), (pts[j, 0], pts[j, 1]), textcoords="offset points", xytext=(5, 5), ha='center')
ax.grid()
legend = ax.legend(legend_handles, legend_labels, facecolor='black')
for text in legend.get_texts():
    text.set_color('white')
ax.set_title(f'TSNE compression for layer {layer}', color='w', fontsize=16)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 12))
nr = 5
nc = 6
for i in range(nr * nc):
    plt.subplot(nr, nc, i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation=None)
    plt.title(f'idx: {i}, truth: {example_targets[i]}', c='w')
    plt.xticks([])
    plt.yticks([])

In [ ]:
example_data.mean(), example_data.std()  # almost at 0.0, 1.0 after normalization

In [ ]:
test_counter

In [ ]:
test() # before training, about 8% accuracy

In [ ]:
with torch.no_grad():
    output = network(example_data)

In [ ]:
fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()), c='w')
    plt.xticks([])
    plt.yticks([])

In [ ]:
# demo of how to keep training from checkpoints:
network_state_dict = torch.load('results/model.pth')
#network = Net()
network = Net2()  # WS mod
network.load_state_dict(network_state_dict)

optimizer_state_dict = torch.load('results/optimizer.pth')
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
optimizer.load_state_dict(optimizer_state_dict)

In [ ]:
n_epochs = 1 # continued training from saved state
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
# WS note: Net2() with 2-neuron bottleneck performance is down to 93% accuracy after 6 epochs,
# compared to 98% accuracy with original network

In [ ]:
with torch.no_grad():
    output = network(example_data)

In [ ]:
len(output)

In [ ]:
for k in range(5):
    print(output[k].argmax().item(), example_targets[k].item())

In [ ]:
test_matrix = precision(test_loader)

In [ ]:
train_matrix = precision(train_loader)

In [ ]:
plt.imshow(np.log(test_matrix + 1))
plt.show()

In [ ]:
plt.imshow(np.log(train_matrix + 1))
plt.show()

In [ ]:
#for k0, k1 in test_loader:
    #print(k0.shape, k1.shape)

## MODIFY THE NETWORK TO INCLUDE A 'BOTTLENECK' 2D EMBEDDING LAYER
## THAT CAN BE PLOTTED DIRECTLY

In [ ]:
# WS model that bottlenecks at 2 neurons, to see if a 2D embedding space can be visualized
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d( 1, 10, kernel_size=5) #  1 channel coming in,  10 channels out
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 10 channels coming in, 20 channels out
        self.conv2_drop = nn.Dropout2d()
        self.fc1   = nn.Linear(320, 50) # fully connected 4 x 4 x 20 = 320 inputs, 50 outputs
        self.emb   = nn.Linear(50, 2)   # WS added to try to get a 2D embedding space
        self.fc2   = nn.Linear(2, 10)   # WS modified to 2 inputs from 50, 10 outputs, 1 for each integer
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.emb(x)  # WS mod
        x = F.dropout(x, training=self.training)  # self.training: Boolean, sets training or eval mode
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # WS added dim=1 to avoid throwing warnings

In [ ]:
network   = Net2()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
n_epochs = 3
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
network

In [ ]:
example_data[0].shape

In [ ]:
network(example_data[0:3])

In [ ]:
example_targets[0:3]

In [ ]:
network

In [ ]:
# test get_intermediate_output()
#ff = get_intermediate_output(network, example_data[0:upper], 'emb')  # chatgpt fails
val   = 4
upper = 200
name = ['conv1', 'conv2', 'conv2_drop', 'fc1', 'emb', 'fc2', 'error_test']
out = get_intermediate(network, name[val], example_data[0:upper])

In [ ]:
#pts   = activation['emb'].numpy()
pts   = out.numpy()
targs = example_targets[0:upper].numpy()

class_colors = {0:'r', 1:'g', 2:'b', 3:'c', 4:'m', 5:'y', 6:'k', 
                7:'#BBBBBB', 8:'#FFAAAA', 9:'#AAFFAA'}

legend_handles = []
legend_labels  = []
for label, color in class_colors.items():
    legend_handles.append(plt.Line2D([], [], marker='o', color='w', markerfacecolor=color, markersize=10))
    legend_labels.append(label)

plt.figure(figsize=(14, 14))
plt.scatter(pts[:,0], pts[:,1], s=10, c=[class_colors[label] for label in targs])
#for j, k in enumerate(targs):
#    plt.annotate(str(k), (pts[j, 0], pts[j, 1]), textcoords="offset points", xytext=(5, 5), ha='center')
plt.grid()
plt.legend(legend_handles, legend_labels)
# turn to a black background